In [ ]:
from random import randint
from datetime import datetime, timedelta

import requests

In [78]:
def build_newsapi_query(entity_name, known_aliases_list, risk_term_list, explicit_entity_name=True):
    if explicit_entity_name:
        entity_name = '"' + entity_name + '"'
    
    entity_part = '({})'.format(' OR '.join([entity_name] + known_aliases_list))
    term_part = '({})'.format(' OR '.join(['"' + term + '"' for term in risk_term_list]))
    
    return '{} AND {}'.format(entity_part, term_part)

In [123]:
DEFAULT_API_TIMEOUT = 30
DAYS_AGO = 500
ENTITY = 'Wells Fargo'
KNOWN_ALIASES_LIST = []
RISK_TERM_LIST = ['fined']#, 'penalty', 'fraud', 'crime']
SORT_BY = 'relevancy'
PAGE_SIZE = 100
LANG = 'en'

NEWS_SOURCES_BATCH_ONE = [
#     'abc-news',
#     'al-jazeera-english',
#     'associated-press',
#     'axios',
#     'bbc-news',
#     'bloomberg',
#     'business-insider',
#     'business-insider-uk',
#     'cbs-news',
#     'cnbc',
#     'cnn',
#     'financial-times',
#     'fortune',
#     'fox-news',
    'google-news',
    'google-news-uk',
#     'independent',
#     'metro',
#     'mirror',
#     'msnbc'
]

NEWS_SOURCES_BATCH_TWO = [
#     'national-review',
#     'nbc-news',
#     'newsweek',
#     'new-york-magazine',
#     'politico',
#     'reddit-r-all',
#     'reuters',
#     'the-american-conservative',
#     'the-economist',
#     'the-guardian-uk',
#     'the-hill',
#     'the-huffington-post',
#     'the-new-york-times',
#     'the-telegraph',
    'the-wall-street-journal',
#     'the-washington-post',
#     'the-washington-times',
#     'time',
#     'usa-today',
#     'vice-news'
 ]

api_key = '774fd37c1f92472ba76cf60ba6099043'

api_url = 'https://newsapi.org/v2/everything'

query = build_newsapi_query(ENTITY, KNOWN_ALIASES_LIST, RISK_TERM_LIST)

params_1 = {
    'q': query,
    'sources': ','.join(NEWS_SOURCES_BATCH_ONE),
#     'from': str(datetime.now() - timedelta(DAYS_AGO)).split()[0],
#     'to': str(datetime.now()).split()[0],
    'language': LANG,
    'sortBy': SORT_BY,
    'pageSize': PAGE_SIZE,
    'page': 1
}

params_2 = {
    'q': query,
    'sources': ','.join(NEWS_SOURCES_BATCH_TWO),
#     'from': str(datetime.now() - timedelta(DAYS_AGO)).split()[0],
#     'to': str(datetime.now()).split()[0],
    'language': LANG,
    'sortBy': SORT_BY,
    'pageSize': PAGE_SIZE,
    'page': 1
}

headers = {
    'Content-Type': 'Application/JSON',
    'Authorization': api_key
}

In [122]:
query

'("Wells Fargo") AND ("fine")'

In [ ]:
def get_newsapi_data(api_url, headers, params, timeout, get_article_list_from_response=lambda r: r.json()['articles']):
    api_results_dict = {}

    while True:
        r = requests.get(api_url, headers=headers, timeout=timeout, params=params)
        
        print("NewsAPISpider received an API response: 'totalResults'={}; 'status'={}".format(r.json()['totalResults'], r.json()['status']))
        
        # handle API errors
        if r.json().get('status', '') == 'error':
            raise Exception("Unexpected error occurred while querying NewsAPI. Code='{}' .. ".format(r.json().get('code', '')))

        article_list = get_article_list_from_response(r)

        if article_list == []:
            print("No articles received in response. Breaking .. ")
            break
        else:
            api_results_dict.update({result['url']: result for result in article_list})
            params['page'] += 1
        
    return api_results_dict

In [124]:
article_dict_batch_1 = get_newsapi_data(api_url, headers, params_1, DEFAULT_API_TIMEOUT)

NewsAPISpider received an API response: 'totalResults'=0; 'status'=ok
No articles received in response. Breaking .. 


In [125]:
article_dict_batch_2 = get_newsapi_data(api_url, headers, params_2, DEFAULT_API_TIMEOUT)

NewsAPISpider received an API response: 'totalResults'=4; 'status'=ok
NewsAPISpider received an API response: 'totalResults'=4; 'status'=ok
No articles received in response. Breaking .. 


In [102]:
total_keys = len(article_dict_batch_1.keys()) + len(article_dict_batch_2.keys())
num_unique_urls = len(set(list(article_dict_batch_1.keys()) + list(article_dict_batch_2.keys())))

In [103]:
total_keys

775

In [104]:
num_unique_urls

775

In [27]:
headers

{'Authorization': '774fd37c1f92472ba76cf60ba6099043',
 'Content-Type': 'Application/JSON'}

In [118]:
article_dict_batch_1

{'http://feedproxy.google.com/~r/TheMoneyGame/~3/LhuABYybdFQ/wells-fargo-stock-price-earnings-per-share-profit-scandal-2018-4': {'source': {'id': 'google-news',
   'name': 'Google News'},
  'author': 'Graham Rapier',
  'title': 'Wells Fargo is climbing after earnings (WFC)',
  'description': 'Wells Fargo topped analysts earnings expectations for the first quarter on Friday. The bank said its results could change depending on the outcome of an investigation into its alleged fraudulent lending practices dating back to 2016. "We also made progress on…',
  'url': 'http://feedproxy.google.com/~r/TheMoneyGame/~3/LhuABYybdFQ/wells-fargo-stock-price-earnings-per-share-profit-scandal-2018-4',
  'urlToImage': None,
  'publishedAt': '2018-04-13T12:37:00Z',
  'content': None}}

In [126]:
article_dict_batch_2

{'https://www.wsj.com/articles/what-went-wrong-with-the-cfpb-1511072512': {'source': {'id': 'the-wall-street-journal',
   'name': 'The Wall Street Journal'},
  'author': 'Dennis Shaul',
  'title': 'What Went Wrong With the CFPB',
  'description': 'I was an aide to Barney Frank. I’ve learned it’s a mistake to create an unaccountable agency.',
  'url': 'https://www.wsj.com/articles/what-went-wrong-with-the-cfpb-1511072512',
  'urlToImage': 'https://si.wsj.net/public/resources/images/ED-AX007_SHAUL_SOC_20171117120635.jpg',
  'publishedAt': '2017-11-19T22:39:54Z',
  'content': 'Richard Cordray’s resignation as director of the Consumer Financial Protection Bureau provides a great opportunity for President Trump to appoint a new director who can undo an unfortunate legacy of bureaucratic overreach and political bias. More important going forward is what we have learned from our experience with the CFPB to prevent future similar missteps. The first lesson is that Congress should never again c

# NewsAPI Source Retrieval

In [32]:
url = "https://newsapi.org/v2/sources?apiKey=774fd37c1f92472ba76cf60ba6099043"
import requests
r = requests.get(url)

sources = [
    item['id'] for item in r.json()['sources'] if 
    item['category'] in ['business', 'general'] and 
    item['language'] in ['en'] and 
    item['country'] in ['us', 'gb'] and
    item['id'] not in ['breitbart-news']
]

In [54]:
sources_batch_one = sources[:20]
sources_batch_two = sources[20:]

In [68]:
sources_batch_one

['abc-news',
 'al-jazeera-english',
 'associated-press',
 'axios',
 'bbc-news',
 'bloomberg',
 'business-insider',
 'business-insider-uk',
 'cbs-news',
 'cnbc',
 'cnn',
 'financial-times',
 'fortune',
 'fox-news',
 'google-news',
 'google-news-uk',
 'independent',
 'metro',
 'mirror',
 'msnbc']

In [67]:
sources_batch_two

['national-review',
 'nbc-news',
 'newsweek',
 'new-york-magazine',
 'politico',
 'reddit-r-all',
 'reuters',
 'the-american-conservative',
 'the-economist',
 'the-guardian-uk',
 'the-hill',
 'the-huffington-post',
 'the-new-york-times',
 'the-telegraph',
 'the-wall-street-journal',
 'the-washington-post',
 'the-washington-times',
 'time',
 'usa-today',
 'vice-news']

# Wayback Date Experiment

In [5]:
import waybackpack
import dateparser

from dateparser.search import search_dates

def get_wayback_date(url):
    try:
        results = waybackpack.search(url, uniques_only=True, collapse='')
    except Exception:
        return None
    
    dates = [dateparser.parse(result['timestamp'], date_formats=['%Y%m%d%H%M%S']) for result in results]
    if dates == []:
        return None
    else:
        return min(dates)

In [6]:
get_wayback_date("https://www.law360.com/articles/1118384/nyse-fines-wedbush-founder-1m-for-unsupervised-trades")

[]


In [105]:
from fuzzywuzzy import fuzz

In [109]:
url1 = "https://www.cnn.com/michael-cohen-testimony-lessons/index.html"
url2 = "https://www.uk.cnn.com/2019/02/27/politics/michael-cohen-testimony-lessons/index.html"
fuzz.ratio(url1, url2)

84

In [2]:
import requests

In [4]:
html = r.text

In [8]:
html

'<!DOCTYPE html>\n<html prefix="og: http://ogp.me/ns#" lang="en">\n    <head>\n        <meta charset="UTF-8" />\n        <meta name="yandex-verification" content="003a3c9d6bff23fb" />\n        <meta name="pocket-site-verification" content="23a18549cf0481f5cbfd0579b6f2bb" />\n        <meta property="fb:pages" content="326683984410" />\n        <meta name="msvalidate.01" content="36879475773792950F4872586A371571" />\n        <meta name="google-site-verification" content="VQLbsaWKiXW2-Vt4lP7MevnhMES537NO4fQjfGgLcbI" />\n        <meta name="apple-mobile-web-app-capable" content="no" />\n        <meta name="format-detection" content="telephone=yes" />\n        <meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=yes" />\n        <meta name="HandheldFriendly" content="true" />\n        <meta name="MobileOptimzied" content="width" />\n        <meta http-equiv="cleartype" content="on" />\n        <meta name="navigation" content="tabbed" />\n        <title>Fined o

In [32]:
from dateparser.search import search_dates
from bs4 import BeautifulSoup
from readability import Document
import requests
import articleDateExtractor

In [172]:
unwanted_parents = [
    'style',
    'script',
    'header',
    'head',
    'title',
    'meta',
    'comment',
    'nav'
]

In [166]:
tags = soup.findAll(attrs={"class": ["header", "offers"]})

In [185]:
type(tags[1].findChildren()[0])

bs4.element.Tag

In [164]:
header_tag.findChildren()[0]

[<div class="header__section header__section_gray">
 <div class="header__content">
 <div class="header__social">
 <ul class="socials">
 <li class="socials__item">
 <a class="socials__link socials__link_facebook" href="//www.facebook.com/RTnews/" target="_blank">facebook</a>
 </li><li class="socials__item">
 <a class="socials__link socials__link_twitter" href="//twitter.com/RT_com" target="_blank">twitter</a>
 </li><li class="socials__item">
 <a class="socials__link socials__link_youtube" href="//www.youtube.com/user/RussiaToday" target="_blank">youtube</a>
 </li><li class="socials__item">
 <a class="socials__link socials__link_instagram" href="//instagram.com/rt" target="_blank">instagram</a>
 </li></ul>
 </div>
 <div class="header__alternative-links">
 <ul class="alternative-links">
 <li class="alternative-links__item">
 <a class="alternative-links__link" href="/applications/" onclick="ga('send', 'event', 'Links', 'Click', 'App Down');">Applications</a></li>
 <li class="alternative-li

In [118]:
el.__repr__()

"'html'"

In [119]:
tags = [tag for tag in tags if 'Feb' in tag.__repr__()]

In [121]:
header_date_el = tags[0]

In [132]:
header_date_el.

('',
 '\n',
 '\t                  \n\t                  18:33 GMT, Feb 28, 2019\n\t            \t')

In [188]:
url = "https://www.rt.com/usa/356790-wells-fargo-settles-student-fraud/"

r = requests.get(url)
# doc = Document(r.text)

# create soup object
soup = BeautifulSoup(r.text)

# extract all visible tags
visible_tags = soup.findAll(text=True)

# find all children of unwanted tags
unwanted_tags = [
    tag for parent_tag_obj in soup.findAll(attrs={"class": unwanted_parents})
    for child_tag_list in parent_tag_obj.findChildren()
    for tag in child_tag_list
]

# filter visible tags for unwanted tag children
visible_tags_filtered = [tag for tag in visible_tags if tag not in unwanted_tags]

print(len(visible_tags))

print(len(visible_tags_filtered))

445
136


In [187]:
str(visible_tags_filtered)

'[\'html\', \'Fined over fees: Wells Fargo to pay $4.1mn for illegal student loans — RT USA News\', \' rev-hash \', \' end \', \'\\n            var googletag = googletag || {};\\n            googletag.cmd = googletag.cmd || [];\\n        \', "\\n        var slots = {\\n            [\'usa_banner\']: {},\\n        };\\n    ", \' EN_RT_USA \', "\\n            googletag.cmd.push(function() {\\n                var mappingLeaderboard = googletag.sizeMapping().\\n                    addSize([1000,0], [[728, 90], [970, 90], [970, 250], [1228, 90]]).\\n                    addSize([768, 0], [[468, 60], [728, 90]]).\\n                    addSize([0, 0], [[300, 50], [320, 50]]).\\n                    build();\\n                var mappingBox = googletag.sizeMapping().\\n                    addSize([1000, 0], [[250, 250], [300, 250], [300, 600], [375, 180], [375, 250], [376, 252]]).\\n                    addSize([768, 0], [[250, 250], [300, 250], [300, 600], [376, 252]]).\\n                    addS

In [91]:
threshold = 100
extracted_text_list = [el.text for el in soup.find_all("div") if len(el.text) < threshold]
extracted_dates = []

In [92]:
extracted_text_list

['',
 '\n\n\nfacebook\n\ntwitter\n\nyoutube\n\ninstagram\n\n',
 '\n\n\nApplications\n\nRSS\n\n ',
 '\n\nالعربيةESPРУСDEFR ',
 '\n\nИНОТВRTДRUPTLY ',
 '',
 '\n\nRTQuestion more\n ',
 '\nlive\n',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\t                  \n\t                  18:11 GMT, Feb 28, 2019\n\t            \t\n\n',
 '\n\n\n\n\n\n\n',
 '\n\n\t                  \n\t                  18:11 GMT, Feb 28, 2019\n\t            \t\n',
 '\n\n\nNewsUSAUKSportRussiaBusinessOp-edRT360ShowsProjects \n',
 '\n\nNewsUSAUKSportRussiaBusinessOp-edRT360ShowsProjects ',
 '\n\n\n\nfacebook\n\ntwitter\n\nyoutube\n\ninstagram\n \n\n',
 '\n\n\nfacebook\n\ntwitter\n\nyoutube\n\ninstagram\n \n',
 '\n',
 'HomeUSA News',
 'HomeUSA News',
 '',
 'Published time: 23 Aug, 2016 01:24 Edited time: 23 Aug, 2016 13:56 ',
 'Get short URL',
 'Get short URL',
 '',
 '',
 '',
 '',
 '',
 '\n',
 'Mixi.Media News',
 '',
 '\n\n \n',
 '\n ',
 '\n\n\n\n\nWhere to watch\n\n\n\n\nSchedule\n\n\n\n',
 '\n\n\n\nWhere to watch\n\n\n\n\nSchedul

In [47]:
d = articleDateExtractor.extractArticlePublishedDate(url, html=doc.html.text_content())

Extracting date from https://www.rt.com/usa/356790-wells-fargo-settles-student-fraud/


In [48]:
d

In [24]:
soup = BeautifulSoup(r.text, "lxml")

In [25]:
soup

<!DOCTYPE html>
<html lang="en" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="003a3c9d6bff23fb" name="yandex-verification"/>
<meta content="23a18549cf0481f5cbfd0579b6f2bb" name="pocket-site-verification"/>
<meta content="326683984410" property="fb:pages"/>
<meta content="36879475773792950F4872586A371571" name="msvalidate.01"/>
<meta content="VQLbsaWKiXW2-Vt4lP7MevnhMES537NO4fQjfGgLcbI" name="google-site-verification"/>
<meta content="no" name="apple-mobile-web-app-capable"/>
<meta content="telephone=yes" name="format-detection"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=yes" name="viewport"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width" name="MobileOptimzied"/>
<meta content="on" http-equiv="cleartype"/>
<meta content="tabbed" name="navigation"/>
<title>Fined over fees: Wells Fargo to pay $4.1mn for illegal student loans — RT USA News</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-ic

In [5]:
search_dates(html)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-453f2b623e8d>", line 1, in <module>
    search_dates(html)
  File "/usr/local/lib/python3.5/dist-packages/dateparser-0.7.0-py3.5.egg/dateparser/search/__init__.py", line 33, in search_dates
    result = _search_with_detection.search_dates(text=text, languages=languages, settings=settings)
  File "/usr/local/lib/python3.5/dist-packages/dateparser-0.7.0-py3.5.egg/dateparser/conf.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.5/dist-packages/dateparser-0.7.0-py3.5.egg/dateparser/search/search.py", line 224, in search_dates
    settings=settings)}
  File "/usr/local/lib/python3.5/dist-packages/dateparser-0.7.0-py3.5.egg/dateparser/search/search.py", line 156, in search_parse
    original=original, translated=translated, settings=setting

KeyboardInterrupt: 

In [1]:
import articleDateExtractor

In [2]:
articleDateExtractor.extractArticlePublishedDate("https://www.insurancejournal.com/news/west/2019/01/02/513382.htm")

datetime.datetime(2019, 1, 2, 0, 0)

In [3]:
from pymongo import MongoClient

In [20]:
client = MongoClient("mongodb://172.24.0.2:27017")

In [21]:
coll = client['negative_news']['scrape_feed']

In [49]:
len(list(coll.find({})))

1337

In [22]:
data = [item for item in coll.find({}) if item['date'] is None]

In [25]:
import pandas as pd
df = pd.DataFrame(data)

In [30]:
pd.set_option('display.max_rows', 500)

In [37]:
t =pd.options.display

In [42]:
t.max_colwidth = 100

In [47]:
len(df['jobname'].unique())

6

In [45]:
df[['entity', 'scraper', 'source', 'title']]

entity        scraper                          source  \
0      Wells Fargo  BingSearchAPI                  Wellsfargo.com   
1      Wells Fargo  BingSearchAPI                En.wikipedia.org   
2      Wells Fargo  BingSearchAPI                  Wellsfargo.com   
3      Wells Fargo  BingSearchAPI                   Bloomberg.com   
4      Wells Fargo  BingSearchAPI                         Bna.com   
5      Wells Fargo  BingSearchAPI                Seekingalpha.com   
6      Wells Fargo  BingSearchAPI   Onlineservices.wellsfargo.com   
7      Wells Fargo  BingSearchAPI                  Abajournal.com   
8      Wells Fargo  BingSearchAPI     Knowledge.wharton.upenn.edu   
9      Wells Fargo  BingSearchAPI    Wellsfargodealerservices.com   
10     Wells Fargo  BingSearchAPI                Wfsettlement.com   
11     Wells Fargo  BingSearchAPI                         Ada.gov   
12     Wells Fargo  BingSearchAPI             Wellsfargomedia.com   
13        Barclays  BingSearchAPI          Banking.barclaysus.com   
14        Barclays  BingSearchAPI                      Dfs.ny.gov   
15        Barclays  BingSearchAPI                   Bloomberg.com   
16        Barclays  BingSearchAPI          Banking.barclaysus.com   
17        Barclays  BingSearchAPI                En.wikipedia.org   
18        Barclays  BingSearchAPI                     Ftfnews.com   
19        Barclays  BingSearchAPI           News.bloomberglaw.com   
20        Barclays  BingSearchAPI                  Barclays.co.uk   
21        Barclays  BingSearchAPI                      Law360.com   
22        Barclays  BingSearchAPI                  Barclays.co.uk   
23        Barclays  BingSearchAPI                   Bloomberg.com   
24        Barclays  BingSearchAPI                      Law360.com   
25        Barclays  BingSearchAPI                        Uber.com   
26        Barclays  BingSearchAPI           Barclaysjewellers.com   
27        Barclays  BingSearchAPI                   Euromoney.com   
28        Barclays  BingSearchAPI               Barclaysrehab.com   
29        Barclays  BingSearchAPI                Creditdonkey.com   
30        Barclays  BingSearchAPI                  Barclays.co.uk   
31        Barclays  BingSearchAPI                  Barclays.co.uk   
32        Barclays  BingSearchAPI               Barclaycardus.com   
33        Barclays  BingSearchAPI         Cards.barclaycardus.com   
34        Barclays  BingSearchAPI                     Barclays.ae   
35        Barclays  BingSearchAPI              Barclayswealth.com   
36        Barclays  BingSearchAPI                   Bloomberg.com   
37        Barclays  BingSearchAPI           Barclayscorporate.com   
38        Barclays  BingSearchAPI                     Youtube.com   
39        Barclays  BingSearchAPI      International.barclays.com   
40        Barclays  BingSearchAPI           Bw.barclaysafrica.com   
41        Barclays  BingSearchAPI           Sc.barclaysafrica.com   
42        Barclays  BingSearchAPI              Mobile.twitter.com   
43        Barclays  BingSearchAPI             Wealth.barclays.com   
44        Barclays  BingSearchAPI              Mobile.twitter.com   
45        Barclays  BingSearchAPI         Cards.barclaycardus.com   
46        Barclays  BingSearchAPI                   Glassdoor.com   
47        Barclays  BingSearchAPI                     Barclays.in   
48        Barclays  BingSearchAPI             Wealth.barclays.com   
49        Barclays  BingSearchAPI                       Ag.ny.gov   
50     Wells Fargo  BingSearchAPI                  Wellsfargo.com   
51     Wells Fargo  BingSearchAPI   Connect.secure.wellsfargo.com   
52     Wells Fargo  BingSearchAPI      Ethicsunwrapped.utexas.edu   
53     Wells Fargo  BingSearchAPI            Thankswellsfargo.com   
54     Wells Fargo  BingSearchAPI           Wffnet.wellsfargo.com   
55     Wells Fargo  BingSearchAPI                    Zellepay.com   
56     Wells Fargo  BingSearchAPI              Mobile.twitter.com   
57     Wel

In [20]:
import re

import articleDateExtractor
from articleDateExtractor import _extractFromURL, _extractFromLDJson, _extractFromMeta, _extractFromHTMLTag
import requests
from bs4 import BeautifulSoup

url = "https://www.rt.com/usa/356790-wells-fargo-settles-student-fraud/"
response = requests.get(url, headers=requests.utils.default_headers())
html = response.text
parsedHTML = BeautifulSoup(html, "lxml")

date = _extractFromURL(url)
print("from URL:", date)

date = _extractFromLDJson(parsedHTML)
print("loaded from JSON:", date)

date = _extractFromMeta(parsedHTML)
print("loaded from meta tag:", date)

date = _extractFromHTMLTag(parsedHTML)
print("loaded from HTML tag:", date)

adr_date = articleDateExtractor.extractArticlePublishedDate(url, html=html)
print("actual returned:", adr_date)

from URL: None
loaded from JSON: None
loaded from meta tag: None
loaded from HTML tag: 2019-03-01 19:32:00+00:00
actual returned: 2019-03-01 19:32:00+00:00


In [7]:
[tag for tag in parsedHTML.find_all(
    ['span', 'p','div'],
    class_=re.compile("pubdate|timestamp|article_date|articledate|date|posted-on|time", re.IGNORECASE))
]

[<div class="header__date">
 <span class="date">
 	                  
 	                  19:21 GMT, Mar 01, 2019
 	            	</span>
 </div>, <span class="date">
 	                  
 	                  19:21 GMT, Mar 01, 2019
 	            	</span>, <div class="article__date"><span class="date date_article-header">Published time: 23 Aug, 2016 01:24 </span><span class="date date_article-header">Edited time: 23 Aug, 2016 13:56 </span></div>, <span class="date date_article-header">Published time: 23 Aug, 2016 01:24 </span>, <span class="date date_article-header">Edited time: 23 Aug, 2016 13:56 </span>]

In [16]:
from dateutil.parser import parse
from dateparser.p

def parseStrDate(dateString):
    try:
        dateTimeObj = parse(dateString)
        return dateTimeObj
    except:
        return None

In [24]:
#<time>
for time in parsedHTML.findAll("time"):
    datetime = time.get('datetime', '')
    if len(datetime) > 0:
        print(parseStrDate(datetime))

    datetime = time.get('class', '')
    if len(datetime) > 0 and datetime[0].lower() == "timestamp":
        print(parseStrDate(time.string))

In [25]:
tag = parsedHTML.find("span", {"itemprop": "datePublished"})
if tag is not None:
    dateText = tag.get("content")
    if dateText is None:
        dateText = tag.text
    if dateText is not None:
        print(parseStrDate(dateText))

In [29]:
#class=
for tag in parsedHTML.find_all(['span', 'p','div'], class_=re.compile("pubdate|timestamp|article_date|articledate|date",re.IGNORECASE)):
    dateText = tag.string
    if dateText is None:
        dateText = tag.text

    possibleDate = parseStrDate(dateText)

    if possibleDate is not None:
        print(possibleDate)

2019-03-01 19:32:00+00:00
2019-03-01 19:32:00+00:00


In [46]:
from dateparser.search import search_dates

def clean_tag_text(tag_text):
    tag_text = tag_text.replace("\n", " ").replace("\t", " ")
    return re.sub("\s+", " ", tag_text).strip()

#class=
date_list = []
for tag in parsedHTML.find_all(['span', 'p','div'], class_=re.compile("pubdate|timestamp|article_date|articledate|date|posted-on",re.IGNORECASE)):
    dateText = tag.string
    if dateText is None:
        dateText = tag.text
    
    dateText = clean_tag_text('"' + dateText + '"')
    
    print("dateText:", dateText)
    
    # possibleDate = parseStrDate(dateText)
    possibleDate = search_dates(dateText)[0][1]
    
    print("possibleDate", possibleDate)

    if possibleDate is not None:
        date_list.append(possibleDate)

if date_list != []:
    print(min(date_list))

dateText: " 19:32 GMT, Mar 01, 2019 "
possibleDate 2019-03-01 19:32:00+00:00
dateText: " 19:32 GMT, Mar 01, 2019 "
possibleDate 2019-03-01 19:32:00+00:00
dateText: "Published time: 23 Aug, 2016 01:24 Edited time: 23 Aug, 2016 13:56 "
possibleDate 2016-08-23 01:24:00
dateText: "Published time: 23 Aug, 2016 01:24 "
possibleDate 2016-08-23 01:24:00
dateText: "Edited time: 23 Aug, 2016 13:56 "
possibleDate 2016-08-23 13:56:00


TypeError: can't compare offset-naive and offset-aware datetimes

In [41]:
date_list[0] - date_list[1]

datetime.timedelta(0)

In [65]:
from datetime import datetime, timezone
test = datetime(date_list[0].year, date_list[0].month, date_list[0].day).replace(tzinfo=timezone.utc)
test

datetime.datetime(2019, 3, 1, 0, 0, tzinfo=datetime.timezone.utc)

In [66]:
type(test)

datetime.datetime

In [61]:
test - date_list[1]

datetime.timedelta(days=-1, seconds=16080)

In [58]:
date_list

[datetime.datetime(2019, 3, 1, 19, 32, tzinfo=<StaticTzInfo 'GMT'>),
 datetime.datetime(2019, 3, 1, 19, 32, tzinfo=<StaticTzInfo 'GMT'>),
 datetime.datetime(2016, 8, 23, 1, 24),
 datetime.datetime(2016, 8, 23, 1, 24),
 datetime.datetime(2016, 8, 23, 13, 56)]